In [40]:
#!pip install openai

In [2]:
from configparser import ConfigParser
import requests
import openai
import json
import os

In [4]:
# Get API key stored in local cfg file
# How to is available here : https://towardsdatascience.com/keeping-credentials-safe-in-jupyter-notebooks-fbd215a8e311

parser = ConfigParser()
_ = parser.read('ObsidianGPT.cfg')
openai.api_key = parser.get('my_api', 'auth_key')

# Function definition

In [5]:
def generate_text(prompt):
    response = openai.ChatCompletion.create(model="gpt-3.5-turbo",messages=[{"role": "user", "content": prompt}])
    return response

In [6]:
def create_obsidian_vault(vault_name, parent_directory):
    vault_path = os.path.join(parent_directory, vault_name)
    
    # Create the vault directory
    os.makedirs(vault_path, exist_ok=True)

    # Create default folders within the vault
    os.makedirs(os.path.join(vault_path, "attachments"), exist_ok=True)
    os.makedirs(os.path.join(vault_path, "notes"), exist_ok=True)
    
    return vault_path

# Create and/or modify a local obsidian vault

In [7]:
vault_name = "MyNewVault"
parent_directory = os.getcwd()
vault_path = create_obsidian_vault(vault_name, parent_directory)

## Prompt created by GPT, asking him to create a prompt to match my input of models

In [ ]:
# Get the list of models for different kinds of data from the GPT API
models_prompt = """
Provide a list of popular machine learning and deep learning models for different types of data, 
grouped by data type and problem type. Include Numerical Data with Regression and Classification models, 
Categorical Data with Classification models, Text Data with Natural Language Processing models, 
Image Data with Computer Vision models and Image Generation models, Time Series Data with Forecasting models, 
Audio Data with Speech Recognition and Music Generation models, Reinforcement Learning with Model-free 
and Model-based Algorithms, and Unstructured Data with Clustering and Dimensionality Reduction models.
"""

In [8]:
# GPT API Call
raw_models_response = generate_text(models_prompt)
models_text = raw_models_response.choices[0].message.content.strip()

## Clean model list

In [10]:
models = []
current_data_type = None

for line in models_text.split('\n'):
    line = line.strip()
    
    if not line:
        continue
    
    if line.endswith(':'):
        current_data_type = line[:-1]
    else:
        model_name = line.lstrip('- ')
        models.append({"name": model_name, "data_type": current_data_type})

## Add data to existing models

In [11]:
for model in models:
    resource_prompt = f"""
    For the {model['name']} model, provide:
    1. A brief description of the model.
    2. The three most relevant use cases.
    3. Three great resources with relevant internet links for implementing the model.
    """
    raw_resource_response = generate_text(resource_prompt)
    resources = raw_resource_response.choices[0].message.content.strip()
    model["resources"] = resources

## Write to Obsidian Vault

In [12]:
for model in models:
    model_filename = f"{model['name']}.md"
    model_filepath = os.path.join(vault_path, model_filename)

    with open(model_filepath, "w") as f:
        f.write(f"**Data Type**: {model['data_type']}\n\n")
        f.write(f"**Description**:\n\n{model['description']}\n\n")

        # Add "See Also" section with links to related models
        f.write(f"**See Also**:\n\n")
        for other_model in models:
            if other_model['name'] != model['name'] and other_model['data_type'] == model['data_type']:
                f.write(f"- [[{other_model['name']}]]\n")

        # Add "Python Resources" section
        f.write(f"**Python Resources**:\n\n{model['resources']}\n\n")

        # Add relevant tags with hierarchy, strip special chars for clarity
        f.write(f"\n---\n")
        root_tag = model['data_type'].replace(' ', '-').lower()
        leaf_tag = model['name'].replace('(', '').replace(')', '').replace(' ', '-').lower()
        f.write(f"tags: #{root_tag}, #{root_tag}/{leaf_tag}\n")
        

KeyError: 'description'